In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import preprocessing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data = pd.read_csv("../input/housesalesprediction/kc_house_data.csv")
print("volume of the data set = {}".format(data.shape))
print(data.head())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
input_columns = ['bedrooms', 'bathrooms', 'floors','grade', 'sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'yr_built']
output_columns = ['price']
relevant_data = data[input_columns+output_columns] #cound add year innovated - year built
print("Input parametrs: 'bedrooms', 'bathrooms', 'floors','grade', 'sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'yr_built'") #[FIXTHIS]
print("Output parametr: \'price\'")
relevant_data.head()

In [ ]:
if True in pd.isnull(relevant_data):
    print("something missing")
else:
    print("nothing is missing")

In [ ]:
sns.pairplot(relevant_data)

In [ ]:
print("Visualising dependency of price on amount of bedrooms:")
sns.scatterplot(x=relevant_data.bedrooms, y=relevant_data.price)
#anomaly detected: over 30 bedrooms doesnt seem sane

In [ ]:
# eliminating anomaly
print("anomaly row:")
print(relevant_data[relevant_data.bedrooms>30])
relevant_data = relevant_data.drop([relevant_data[relevant_data.bedrooms>30].index[0]], inplace = False)

In [ ]:
print("Visualising dependency of price on amount of bathrooms:")
sns.scatterplot(relevant_data.bathrooms, relevant_data.price)

In [ ]:
print("Visualising dependency of price on amount of floors:")
sns.scatterplot(relevant_data.floors, relevant_data.price)

In [ ]:
print("Visualising dependency of price on grade:")
sns.scatterplot(relevant_data.grade, relevant_data.price)

In [ ]:
print("Visualising dependency of price on sqft_living:")
sns.scatterplot(relevant_data.sqft_living, relevant_data.price)

In [ ]:
print("Visualising dependency of price on sqft_lot:")
sns.scatterplot(relevant_data.sqft_lot, relevant_data.price)

In [ ]:
print("Visualising dependency of price on sqft_living15:")
sns.scatterplot(relevant_data.sqft_living15, relevant_data.price)

In [ ]:
print("Visualising dependency of price on sqft_lot15:")
sns.scatterplot(relevant_data.sqft_lot15, relevant_data.price)

In [ ]:
print("Visualising dependency of price on yr_built:")
sns.scatterplot(relevant_data.yr_built, relevant_data.price)
input_columns.remove("yr_built")
relevant_data = relevant_data.drop(columns = "yr_built")

Заметим, что от года постройки цена зависит как-то сомнительно: старые дома, построенные в первой половине 20 ст. выставлены на продажу по солидным ценам, сравнимым по порядку с ценами новых домов. Только лишь немногие дома новые(1980+) дома заментно отличаются по цене.
По этому уберираем цену из признаков, по которым строим модель

In [ ]:
#spliting to test/train 
train_data, test_data = train_test_split(relevant_data, test_size = 0.5, random_state = 666)
print("train data set:\n{}".format(train_data))
print("test data set:\n{}".format(test_data))

In [ ]:
##Normalizer
#Normalize data, [maybe use StandardScaler]
#transformer = preprocessing.Normalizer().fit(train_data)
# #MinMaxScaler
# transformer = preprocessing.MinMaxScaler().fit(train_data)
# scaled_train_data, scaled_test_data = transformer.transform(train_data), transformer.transform(test_data)

print(np.arange(scaled_train_data.shape[1]-1))
print(scaled_train_data.shape)
print(scaled_train_data)
print(scaled_train_data[:, np.arange(scaled_train_data.shape[1]-1)])

train_data_x = scaled_train_data[:, np.arange(scaled_train_data.shape[1]-1)]
train_data_y = scaled_train_data[:,-1].reshape(-1, 1)
test_data_x = scaled_test_data[:, np.arange(scaled_test_data.shape[1]-1)]
test_data_y = scaled_test_data[:,-1].reshape(-1, 1)
print(scaled_train_data[-1])
print(train_data_x.shape)
print(train_data_y.shape)
#train_data_x, train_data_y, test_data_x, test_data_y = transformer.transform(train_data[input_columns]), transformer.transform(train_data.price.values.reshape(-1, 1)), transformer.transform(test_data[input_columns]),transformer.transform(test_data.price.values.reshape(-1, 1))
#train_data_x, train_data_y, test_data_x, test_data_y = train_data[input_columns], train_data.price, test_data[input_columns], test_data.price
print("train data set x :\n{}".format(train_data_x))
print("train data set y :\n{}".format(train_data_y))
print("test data set x :\n{}".format(test_data_x))
print("test data set y :\n{}".format(test_data_y))

In [ ]:
#lasso
lasso_model = linear_model.Lasso(alpha = 0.000001, normalize = False, max_iter = 10000000, tol = 0.00000001)
lasso_model.fit(train_data_x, train_data_y)
lasso_results = lasso_model.predict(test_data_x)
print("LASSO results:")
print("train accuracy = {}".format(lasso_model.score(train_data_x, train_data_y)))
print("test accuracy = {}".format(lasso_model.score(test_data_x, test_data_y)))

In [ ]:
#ridge
ridge_model = linear_model.Ridge(alpha = 0.000001, normalize = False, max_iter = 10000000, tol = 0.00000001)
ridge_model.fit(train_data_x, train_data_y)
ridge_results = ridge_model.predict(test_data_x)
print("Ridge results:")
print("train accuracy = {}".format(ridge_model.score(train_data_x, train_data_y)))
print("test accuracy = {}".format(ridge_model.score(test_data_x, test_data_y)))

In [ ]:
#elasticNet
elasticNet_model = linear_model.ElasticNet(alpha = 0.000001, normalize = False, max_iter = 10000000, tol = 0.00000001)
elasticNet_model.fit(train_data_x, train_data_y)
elasticNet_results = elasticNet_model.predict(test_data_x)
print("ElasticNet results:")
print("train accuracy = {}".format(ridge_model.score(train_data_x, train_data_y)))
print("test accuracy = {}".format(ridge_model.score(test_data_x, test_data_y)))

Видим, что все 3 метода генерируют модели с приблизительно одинаковой точностью предсказывания. Можно сделать вывод, что модель система недоопределена(нужно больше данных или меньше параметров)